In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Data Science /Penambangan Data (Data Mining)/Unsupervised Learning/Associan Rules/Tugas 4 Associan Rules/Data1.csv')
data.head()

,Timestamp,Nama,Minuman kekinian apa saja yang sering Anda konsumsi?
0,12/26/2025 20:53:10,anggit,"Kopi Susu, Thai Tea, Matcha latte, Kopi gula a..."
1,12/26/2025 20:53:20,Aszra,"Kopi Susu, Thai Tea, Kopi gula aren"
2,12/26/2025 20:54:20,Isthi,Yakult drink
3,12/26/2025 20:56:03,Akbar,"Kopi Susu, Yakult drink, Chocolate latte"
4,12/26/2025 20:57:58,Yaqin,"Kopi Susu, Yakult drink"


In [3]:
print(data.columns)

Index(['Timestamp', 'Nama',
       'Minuman kekinian apa saja yang sering Anda konsumsi?'],
      dtype='object')


# **Pre-Processing**

**Pengambilan Kolom Minuman**

In [4]:
kolom_minuman ='Minuman kekinian apa saja yang sering Anda konsumsi?'
data_minuman = data[kolom_minuman]

*Karena fungsi association rule bertujuan untuk menemukan pola keterkaitan dan rekomendasi antar barang berdasarkan kemunculan item dalam satu transaksi, maka atribut yang digunakan dalam analisis ini dibatasi pada kolom minuman sebagai representasi item yang dikonsumsi oleh responden.*

In [5]:
data_minuman.head()

,Minuman kekinian apa saja yang sering Anda konsumsi?
0,"Kopi Susu, Thai Tea, Matcha latte, Kopi gula a..."
1,"Kopi Susu, Thai Tea, Kopi gula aren"
2,Yakult drink
3,"Kopi Susu, Yakult drink, Chocolate latte"
4,"Kopi Susu, Yakult drink"


**Split Data**

In [6]:
data_minuman = (data_minuman
                .str.split(',')
)

*Karena data minuman awalnya berbentuk string mentah dengan pemisah berupa tanda koma (,), maka diperlukan proses pemisahan (split) untuk mengubah setiap respons menjadi kumpulan item per transaksi. Tahapan ini dilakukan agar data dapat ditransformasikan ke dalam bentuk numerik melalui proses encoding sebagai dasar analisis association rules.*

In [7]:
data_minuman.head()

,Minuman kekinian apa saja yang sering Anda konsumsi?
0,"[Kopi Susu, Thai Tea, Matcha latte, Kopi gu..."
1,"[Kopi Susu, Thai Tea, Kopi gula aren]"
2,[Yakult drink]
3,"[Kopi Susu, Yakult drink, Chocolate latte]"
4,"[Kopi Susu, Yakult drink]"


In [8]:
data_minuman_clean = [
    [item.strip() for item in transaksi]
    for transaksi in data_minuman
]

*Karena data yang digunakan dalam penelitian ini diperoleh dari hasil kuesioner, terdapat ketidakkonsistenan pada penulisan data responden, khususnya perbedaan penggunaan spasi dan format penamaan item minuman.*

**Encoding**

In [9]:
te = TransactionEncoder()
te_array = te.fit(data_minuman_clean).transform(data_minuman_clean)

df_encoded = pd.DataFrame(te_array, columns=te.columns_)

*Proses encoding dilakukan untuk mentransformasikan data kategorikal ke dalam bentuk numerik (integer/biner) sehingga setiap kategori dapat dianalisis secara komputasional pada tahap selanjutnya*

In [10]:
df_encoded.head()

,Choco Oreo,Chocolate latte,Kopi Susu,Kopi gula aren,Mango yakult,Matcha latte,Milk Tea,Taro latte,Thai Tea,Yakult drink
0,True,False,True,True,False,True,False,False,True,False
1,False,False,True,True,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,True
3,False,True,True,False,False,False,False,False,False,True
4,False,False,True,False,False,False,False,False,False,True


In [11]:
(df_encoded == 1).sum()

,0
Choco Oreo,8
Chocolate latte,4
Kopi Susu,6
Kopi gula aren,6
Mango yakult,4
Matcha latte,5
Milk Tea,3
Taro latte,3
Thai Tea,4
Yakult drink,5


# **FP_Growth**

In [12]:
from mlxtend.frequent_patterns import fpgrowth
#FP_Growth
frequent_itemsets = fpgrowth(
    df_encoded,
    min_support=0.1,
    use_colnames=True
)

#generate rules
rules_fp = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.7
)

print(rules_fp[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                      antecedents       consequents  support  confidence  \
0                  (Matcha latte)      (Choco Oreo)     0.20        0.80   
1      (Choco Oreo, Matcha latte)  (Kopi gula aren)     0.15        0.75   
2    (Choco Oreo, Kopi gula aren)    (Matcha latte)     0.15        1.00   
3  (Matcha latte, Kopi gula aren)      (Choco Oreo)     0.15        1.00   
4           (Thai Tea, Kopi Susu)  (Kopi gula aren)     0.10        1.00   
5      (Thai Tea, Kopi gula aren)       (Kopi Susu)     0.10        1.00   

       lift  
0  2.000000  
1  2.500000  
2  4.000000  
3  2.500000  
4  3.333333  
5  3.333333  


In [13]:
from mlxtend.frequent_patterns import fpgrowth
#FP_Growth
frequent_itemsets = fpgrowth(
    df_encoded,
    min_support=0.1,
    use_colnames=True
)

#generate rules
rules_fp = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.5
)

print(rules_fp[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                       antecedents                   consequents  support  \
0                 (Kopi gula aren)                  (Choco Oreo)     0.15   
1                      (Kopi Susu)              (Kopi gula aren)     0.20   
2                 (Kopi gula aren)                   (Kopi Susu)     0.20   
3                     (Choco Oreo)                (Matcha latte)     0.20   
4                   (Matcha latte)                  (Choco Oreo)     0.20   
5                   (Matcha latte)              (Kopi gula aren)     0.15   
6                 (Kopi gula aren)                (Matcha latte)     0.15   
7       (Choco Oreo, Matcha latte)              (Kopi gula aren)     0.15   
8     (Choco Oreo, Kopi gula aren)                (Matcha latte)     0.15   
9   (Matcha latte, Kopi gula aren)                  (Choco Oreo)     0.15   
10                  (Matcha latte)  (Choco Oreo, Kopi gula aren)     0.15   
11                (Kopi gula aren)    (Choco Oreo, Matcha latte)     0.15   

In [14]:
from mlxtend.frequent_patterns import fpgrowth
#FP_Growth
frequent_itemsets = fpgrowth(
    df_encoded,
    min_support=0.2,
    use_colnames=True
)

#generate rules
rules_fp = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.5
)

print(rules_fp[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

        antecedents       consequents  support  confidence      lift
0       (Kopi Susu)  (Kopi gula aren)      0.2    0.666667  2.222222
1  (Kopi gula aren)       (Kopi Susu)      0.2    0.666667  2.222222
2      (Choco Oreo)    (Matcha latte)      0.2    0.500000  2.000000
3    (Matcha latte)      (Choco Oreo)      0.2    0.800000  2.000000
